In [38]:
import pandas as pd
import oanda_utils
import plotly.graph_objects as go

In [39]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c, line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))

    for bb_point in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter(
            x=df_plot.time,
            y=df_plot[bb_point],
            line=dict(width=2, color='#4290F5'),
            line_shape='spline',
            name=bb_point,
            mode='lines'
            ))

    df_markers = df_plot[df_plot.BB_SIGNAL != 0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c, line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor='#3480eb',
        increasing_line_color='#3480eb',
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000, height=400, margin=dict(l=10,r=10,b=10,t=10), font=dict(size=10, color='#e1e1e1'), paper_bgcolor='#1e1e1e', plot_bgcolor='#1e1e1e')
    fig.update_xaxes(
        gridcolor='#1F292F', showgrid=True, fixedrange=True, rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor='#1F292F', 
        showgrid=True
    )
    fig.show()

In [40]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(oanda_utils.get_his_data_filename(pair, granularity))

In [41]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [42]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [43]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [44]:
df_plot = df.iloc[700:800]
plot_candles(df_plot)

In [45]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)


In [46]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
75,2018-01-08 02:00:00+00:00,113.128,113.274,113.114,113.230,113.206000,112.738600,0.335957,113.410514,112.066686,0,112.841058,112.994199,0.153142,0.153142,0.000000
78,2018-01-08 06:00:00+00:00,113.232,113.387,113.119,113.162,113.222667,112.785417,0.334854,113.455125,112.115709,0,112.868233,113.020302,0.152068,0.152545,-0.000477
81,2018-01-08 10:00:00+00:00,113.159,113.167,112.884,112.930,112.993667,112.819400,0.318587,113.456574,112.182226,0,112.873408,113.006279,0.132870,0.144482,-0.011612
84,2018-01-08 14:00:00+00:00,112.928,113.125,112.904,113.082,113.037000,112.855717,0.298295,113.452306,112.259127,0,112.890717,113.018020,0.127303,0.138662,-0.011359
87,2018-01-08 18:00:00+00:00,113.084,113.126,113.026,113.092,113.081333,112.897200,0.265788,113.428777,112.365623,0,112.907272,113.029478,0.122206,0.133767,-0.011561


In [51]:
def plot_bb(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2, color='#03adfc'),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))

    fig.add_trace(go.Scatter(
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2, color='#fc4a03'),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))

    fig.add_trace(go.Bar(
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))

    fig.update_layout(width=1000, height=400, margin=dict(l=10,r=10,b=10,t=10), font=dict(size=10, color='#e1e1e1'), paper_bgcolor='#1e1e1e', plot_bgcolor='#1e1e1e')
    fig.update_xaxes(
        gridcolor='#1F292F', showgrid=True, fixedrange=True, rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor='#1F292F', 
        showgrid=True
    )
    fig.show()

In [53]:
df_plot = df.iloc[1000:1200]
plot_bb(df_plot)